In [115]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [116]:
data = pd.read_csv('new_combined_data.csv')

In [117]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [118]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [119]:
data.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description', 'num_words_in_transcript',
       'resume_jd_similarity', 'resume_transcript_similarity', 'sentiment',
       'polarity', 'lexical_diversity', 'transcript_length',
       'technical_skill_match', 'soft_skills_sentiment', 'resume_length',
       'job_description_experience_match', 'cultural_fit_sentiment',
       'job_fit_score', 'confidence_score', 'job_desc_complexity',
       'interaction_quality', 'clarity_score', 'text_complexity_transcript',
       'text_complexity_resume'],
      dtype='object')

In [120]:
# Custom Dataset for Batch Processing
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Function to get embeddings batch-wise
def generate_embeddings(texts, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    embeddings = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            encoded_inputs = tokenizer(
                list(batch), return_tensors='pt', truncation=True, padding=True, max_length=512
            )
            encoded_inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
            outputs = model(**encoded_inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(batch_embeddings)

    return np.vstack(embeddings)


In [121]:
text_features = ['Reason for decision', 'Job Description', 'polarity']
numerical_features = [
    'num_words_in_transcript', 'resume_jd_similarity', 
    'resume_transcript_similarity', 'sentiment',
    'lexical_diversity', 'transcript_length', 'technical_skill_match',
    'soft_skills_sentiment', 'resume_length',
    'job_description_experience_match', 'cultural_fit_sentiment',
    'job_fit_score', 'confidence_score', 'job_desc_complexity',
    'interaction_quality', 'clarity_score', 
    'text_complexity_transcript', 'text_complexity_resume'
]

In [122]:
# Generate embeddings
for feature in text_features:
    print(f"Generating embeddings for {feature}...")
    data[f'{feature}_embedding'] = list(generate_embeddings(data[feature].tolist()))

print("All embeddings generated successfully.")

# Normalize numerical features
scaler = MinMaxScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Concatenate all features
embeddings = np.concatenate(
    [np.vstack(data[f'{feat}_embedding'].to_numpy()) for feat in text_features] +
    [data[numerical_features].to_numpy()], axis=1
)

Generating embeddings for Reason for decision...
Generating embeddings for Job Description...
Generating embeddings for polarity...
All embeddings generated successfully.


In [123]:
# Save concatenated embeddings and features back to the dataset
embedding_df = pd.DataFrame(
    embeddings, 
    columns=[f"feature_{i}" for i in range(embeddings.shape[1])]
)

# Add the target column back for supervised learning
embedding_df['decision'] = data['decision'].values

In [151]:
embedding_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_2313,feature_2314,feature_2315,feature_2316,feature_2317,feature_2318,feature_2319,feature_2320,feature_2321,decision
0,0.179324,0.094607,-0.209987,0.082771,0.817316,-0.448502,0.083290,0.849484,-0.000335,-0.337400,...,0.833333,0.333333,0.076585,0.461538,0.504845,0.613356,0.772682,0.435219,0.370576,0
1,-0.225972,-0.208748,0.135149,0.150673,0.485873,0.055903,0.022488,0.389991,-0.174321,-0.719327,...,0.391667,0.333333,0.210348,0.076923,0.779988,0.768511,0.299537,0.401584,0.429927,1
2,0.049797,0.304285,-0.228528,0.102754,0.240614,0.208117,0.364976,0.522812,-0.010486,-0.389484,...,0.858333,0.333333,0.072764,0.230769,0.703692,0.582768,0.752163,0.312548,0.469971,0
3,0.095022,-0.180992,0.011591,0.011805,-0.035110,-0.005788,0.106368,0.139976,0.018789,-0.342472,...,0.750000,0.540741,0.164754,0.000000,0.372348,0.688049,0.543351,0.414987,0.483559,1
4,0.088482,0.300307,-0.178998,0.007200,0.475154,0.168386,0.281142,0.566269,-0.050585,-0.304064,...,0.725000,0.600000,0.060762,0.307692,0.449969,0.632492,0.533092,0.440336,0.607910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,0.233737,0.078628,-0.030567,0.054535,0.193983,0.054798,0.351920,-0.021288,-0.061951,-0.344478,...,0.133333,0.333333,0.516057,0.153846,0.770451,0.729232,0.555623,0.399964,0.541649,1
3170,0.231737,0.046225,-0.256670,0.001107,0.293122,-0.129435,-0.024141,0.317838,-0.032955,-0.186927,...,0.150000,0.333333,0.591477,0.307692,0.762699,0.699965,0.303762,0.313024,0.534925,0
3171,0.297704,-0.047423,-0.014681,0.033673,0.027335,-0.071068,0.320845,-0.013860,-0.027852,-0.193233,...,0.166667,0.500000,0.552929,0.000000,0.760098,0.803279,0.533092,0.351979,0.506818,1
3172,0.332778,0.196317,-0.184458,0.044037,0.461602,-0.142836,0.100957,0.141142,-0.057379,-0.298707,...,0.141667,0.333333,0.556526,0.076923,0.767901,0.740856,0.352645,0.345215,0.402437,0


In [152]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import optuna

# Encode target variable
y = data['decision']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)


In [126]:
# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define the objective function
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    # Silent training with verbose=0
    model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)

    # Print ROC and AUC score for the current trial
    print(f"Trial completed - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")
    return roc_auc  # Optimize for AUC

# Run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best parameters and AUC
print("\nBest parameters:", study.best_params)
print("Best ROC-AUC score:", study.best_value)


Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8883
Trial completed - Accuracy: 0.8866, ROC-AUC: 0.8868
Trial completed - Accuracy: 0.8850, ROC-AUC: 0.8852
Trial completed - Accuracy: 0.8709, ROC-AUC: 0.8711
Trial completed - Accuracy: 0.8850, ROC-AUC: 0.8851
Trial completed - Accuracy: 0.8850, ROC-AUC: 0.8852
Trial completed - Accuracy: 0.8835, ROC-AUC: 0.8836
Trial completed - Accuracy: 0.8850, ROC-AUC: 0.8852
Trial completed - Accuracy: 0.8835, ROC-AUC: 0.8836
Trial completed - Accuracy: 0.8835, ROC-AUC: 0.8836

Best parameters: {'max_depth': 10, 'learning_rate': 0.12884503793747262, 'n_estimators': 556, 'subsample': 0.5138162711002101, 'colsample_bytree': 0.6670832638580284, 'gamma': 0.6181850881559919, 'reg_alpha': 5.205493132063401, 'reg_lambda': 3.9857976272211437}
Best ROC-AUC score: 0.8883278441331693


In [127]:
# Best parameters and final model training
best_params = study.best_params
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'max_depth': 10, 'learning_rate': 0.12884503793747262, 'n_estimators': 556, 'subsample': 0.5138162711002101, 'colsample_bytree': 0.6670832638580284, 'gamma': 0.6181850881559919, 'reg_alpha': 5.205493132063401, 'reg_lambda': 3.9857976272211437}


In [128]:
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
final_model.fit(X_train, y_train)

c:\Users\Durgesh Babu\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:22:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6670832638580284, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.6181850881559919, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.12884503793747262,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=556, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [129]:
# Predictions and evaluation
y_pred = final_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC_AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.8881889763779528
ROC_AUC: 0.8883278441331693


In [153]:
y_test_pred_classes = final_model.predict_proba(X_test)
y_test_pred_xgb_distil = np.argmax(y_test_pred_classes, axis=1)

In [131]:
from sentence_transformers import SentenceTransformer

In [132]:
# Define a custom dataset for efficient DataLoader usage
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# Function to generate embeddings using CLS Token Pooling with SBERT
def generate_embeddings(texts, model_name='all-MiniLM-L6-v2', batch_size=32, max_length=512):
    """
    Generates embeddings using Sentence Transformers with CLS token pooling.
    Args:
        texts (list): List of texts to embed.
        model_name (str): Pre-trained SentenceTransformer model.
        batch_size (int): Batch size for embedding generation.
        max_length (int): Maximum token length for each text.
    Returns:
        np.ndarray: Generated embeddings.
    """
    # Load the SentenceTransformer model
    model = SentenceTransformer(model_name)
    model.max_seq_length = max_length  # Adjust max token length

    # Dataset and DataLoader for batch processing
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()

    with torch.no_grad():
        for batch in dataloader:
            # Tokenize and encode with CLS token pooling
            batch_embeddings = model.encode(
                batch, 
                batch_size=batch_size,
                convert_to_tensor=True,
                show_progress_bar=False,
                normalize_embeddings=True  # Ensures cosine similarity compatibility
            )
            embeddings.append(batch_embeddings.cpu().numpy())

    return np.vstack(embeddings)

In [133]:
text_features = ['Reason for decision', 'Job Description', 'polarity']
numerical_features = [
    'num_words_in_transcript', 'resume_jd_similarity', 
    'resume_transcript_similarity', 'sentiment',
    'lexical_diversity', 'transcript_length', 'technical_skill_match',
    'soft_skills_sentiment', 'resume_length',
    'job_description_experience_match', 'cultural_fit_sentiment',
    'job_fit_score', 'confidence_score', 'job_desc_complexity',
    'interaction_quality', 'clarity_score', 
    'text_complexity_transcript', 'text_complexity_resume'
]

In [134]:
# Generate embeddings
for feature in text_features:
    print(f"Generating embeddings for {feature}...")
    data[f'{feature}_embedding'] = list(generate_embeddings(data[feature].tolist()))

print("All embeddings generated successfully.")

# Normalize numerical features
scaler = MinMaxScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Concatenate all features
embeddings1 = np.concatenate(
    [np.vstack(data[f'{feat}_embedding'].to_numpy()) for feat in text_features] +
    [data[numerical_features].to_numpy()], axis=1
)

Generating embeddings for Reason for decision...
Generating embeddings for Job Description...
Generating embeddings for polarity...
All embeddings generated successfully.


In [136]:
# Save concatenated embeddings and features back to the dataset
embedding_df1 = pd.DataFrame(
    embeddings1, 
    columns=[f"feature_{i}" for i in range(embeddings1.shape[1])]
)

# Add the target column back for supervised learning
embedding_df1['decision'] = data['decision'].values

In [154]:
embedding_df1

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1161,feature_1162,feature_1163,feature_1164,feature_1165,feature_1166,feature_1167,feature_1168,feature_1169,decision
0,-0.040162,-0.017108,-0.045824,-0.001299,0.042042,-0.049487,-0.006759,-0.037472,-0.010179,-0.061392,...,0.833333,0.333333,0.076585,0.461538,0.504845,0.613356,0.772682,0.435219,0.370576,0
1,0.068834,0.011094,-0.033867,-0.043404,0.028510,-0.001021,0.003934,0.019694,-0.009958,0.027810,...,0.391667,0.333333,0.210348,0.076923,0.779988,0.768511,0.299537,0.401584,0.429927,1
2,0.102072,-0.001394,0.055262,0.090628,-0.005320,-0.061961,0.004546,0.049599,0.011135,-0.049618,...,0.858333,0.333333,0.072764,0.230769,0.703692,0.582768,0.752163,0.312548,0.469971,0
3,0.037205,0.082585,-0.006266,0.015953,-0.021017,-0.007450,0.005732,-0.063339,-0.065506,-0.040845,...,0.750000,0.540741,0.164754,0.000000,0.372348,0.688049,0.543351,0.414987,0.483559,1
4,0.063624,-0.021584,0.045728,0.059596,0.067983,-0.040720,-0.034877,0.012269,0.008448,-0.004602,...,0.725000,0.600000,0.060762,0.307692,0.449969,0.632492,0.533092,0.440336,0.607910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,-0.004482,0.006377,-0.001969,0.027536,0.059272,0.013968,0.021843,0.012360,-0.031767,-0.085682,...,0.133333,0.333333,0.516057,0.153846,0.770451,0.729232,0.555623,0.399964,0.541649,1
3170,-0.041130,0.039581,-0.000266,-0.037578,-0.073201,0.051874,0.152850,-0.010868,0.022315,-0.046826,...,0.150000,0.333333,0.591477,0.307692,0.762699,0.699965,0.303762,0.313024,0.534925,0
3171,0.050766,0.094263,-0.019963,0.043912,0.015446,-0.005203,0.037215,-0.071231,-0.046503,-0.045416,...,0.166667,0.500000,0.552929,0.000000,0.760098,0.803279,0.533092,0.351979,0.506818,1
3172,-0.002540,0.007101,0.002338,0.001370,-0.082722,-0.019490,0.161442,0.091036,-0.031743,0.054705,...,0.141667,0.333333,0.556526,0.076923,0.767901,0.740856,0.352645,0.345215,0.402437,0


In [155]:
# Encode target variable
y = data['decision']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(embeddings1, y, test_size=0.2, random_state=42)

In [139]:
# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Define the objective function
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    # Silent training with verbose=0
    model = xgb.XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)

    # Print ROC and AUC score for the current trial
    print(f"Trial completed - Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}")
    return roc_auc  # Optimize for AUC

# Run the study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best parameters and AUC
print("\nBest parameters:", study.best_params)
print("Best ROC-AUC score:", study.best_value)


Trial completed - Accuracy: 0.8772, ROC-AUC: 0.8773
Trial completed - Accuracy: 0.8803, ROC-AUC: 0.8804
Trial completed - Accuracy: 0.8787, ROC-AUC: 0.8788
Trial completed - Accuracy: 0.9008, ROC-AUC: 0.9009
Trial completed - Accuracy: 0.8803, ROC-AUC: 0.8804
Trial completed - Accuracy: 0.8976, ROC-AUC: 0.8977
Trial completed - Accuracy: 0.8913, ROC-AUC: 0.8914
Trial completed - Accuracy: 0.8929, ROC-AUC: 0.8930
Trial completed - Accuracy: 0.8882, ROC-AUC: 0.8883
Trial completed - Accuracy: 0.8913, ROC-AUC: 0.8915

Best parameters: {'max_depth': 3, 'learning_rate': 0.2661074821620344, 'n_estimators': 950, 'subsample': 0.9340447046791226, 'colsample_bytree': 0.6817068107732505, 'gamma': 1.1716834893600607, 'reg_alpha': 3.3676825424830605, 'reg_lambda': 6.966601507154071}
Best ROC-AUC score: 0.9008819094480378


In [140]:
best_params = study.best_params
final_model2 = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss')
final_model2.fit(X_train, y_train)

c:\Users\Durgesh Babu\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [00:45:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6817068107732505, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=1.1716834893600607, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2661074821620344,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=950, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [141]:
# Predictions and evaluation
y_pred = final_model2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC_AUC:", roc_auc_score(y_test, y_pred))

Accuracy: 0.9007874015748032
ROC_AUC: 0.9008819094480378


In [156]:
y_test_pred_classes = final_model2.predict_proba(X_test)
y_test_pred_xgb_sen= np.argmax(y_test_pred_classes, axis=1)

ANN

In [163]:
import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Function to create and train the model for Optuna optimization
def objective(trial):
    model = Sequential()
    
    # Hyperparameter tuning for number of units and layers
    units_1 = trial.suggest_int('units_1', 32, 256, step=32)
    dropout_1 = trial.suggest_float('dropout_1', 0.2, 0.5, step=0.1)
    units_2 = trial.suggest_int('units_2', 32, 128, step=32)
    dropout_2 = trial.suggest_float('dropout_2', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    # Build the model
    model.add(Dense(units=units_1, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(rate=dropout_1))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(rate=dropout_2))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), verbose=0)
    
    # Evaluate the model on validation data
    score = model.evaluate(X_test, y_test, verbose=0)
    return score[1]  # Return validation accuracy

# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Get the best parameters and train the final model
best_params = study.best_params
print("Best Hyperparameters:", best_params)



C:\Users\Durgesh Babu\AppData\Local\Temp\ipykernel_11308\2231157618.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


Best Hyperparameters: {'units_1': 32, 'dropout_1': 0.30000000000000004, 'units_2': 64, 'dropout_2': 0.4, 'learning_rate': 0.001377913632373173}


In [164]:
# Final model using the best parameters
final_model1 = Sequential()
final_model1.add(Dense(units=best_params['units_1'], activation='relu', input_dim=X_train.shape[1]))
final_model1.add(Dropout(rate=best_params['dropout_1']))
final_model1.add(Dense(units=best_params['units_2'], activation='relu'))
final_model1.add(Dropout(rate=best_params['dropout_2']))
final_model1.add(Dense(1, activation='sigmoid'))

final_model1.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), 
                    loss='binary_crossentropy', 
                    metrics=['accuracy'])

# Train the final model
final_model1.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the final model
final_accuracy = final_model1.evaluate(X_test, y_test)
print(f"Final Model Test Accuracy: {final_accuracy[1]:.4f}")


Epoch 1/100
159/159 [==============================] - 1s 4ms/step - loss: 0.4315 - accuracy: 0.7987 - val_loss: 0.2211 - val_accuracy: 0.8772
Epoch 2/100
159/159 [==============================] - 0s 3ms/step - loss: 0.2051 - accuracy: 0.8885 - val_loss: 0.1658 - val_accuracy: 0.8945
Epoch 3/100
159/159 [==============================] - 0s 3ms/step - loss: 0.1710 - accuracy: 0.8944 - val_loss: 0.1622 - val_accuracy: 0.8835
Epoch 4/100
159/159 [==============================] - 0s 3ms/step - loss: 0.1665 - accuracy: 0.8881 - val_loss: 0.1608 - val_accuracy: 0.8882
Epoch 5/100
159/159 [==============================] - 0s 3ms/step - loss: 0.1607 - accuracy: 0.8901 - val_loss: 0.1569 - val_accuracy: 0.9024
Epoch 6/100
159/159 [==============================] - 1s 3ms/step - loss: 0.1585 - accuracy: 0.8933 - val_loss: 0.1641 - val_accuracy: 0.8740
Epoch 7/100
159/159 [==============================] - 0s 3ms/step - loss: 0.1577 - accuracy: 0.8988 - val_loss: 0.1578 - val_accuracy: 0.8803

In [165]:
y_test_pred_nn = final_model1.predict(X_test)

20/20 [==============================] - 0s 2ms/step


In [166]:
test_df = pd.DataFrame()
test_df['actual'] = y_test
test_df['xg_distil_bert'] = y_test_pred_xgb_distil
test_df['xg_sen_transformer'] = y_test_pred_xgb_sen
test_df['nn_prediction'] = y_test_pred_nn

In [167]:
test_df

,actual,xg_distil_bert,xg_sen_transformer,nn_prediction
2270,1,1,1,1.000000e+00
442,1,1,1,1.000000e+00
2885,1,1,1,1.000000e+00
1655,1,0,1,3.889574e-01
1001,0,0,0,4.208729e-22
...,...,...,...,...
282,0,0,0,1.052493e-18
1953,1,1,1,3.498156e-01
691,0,0,0,8.075190e-19
794,0,0,0,6.212544e-19


In [168]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def calculate_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1-Score': f1_score(y_true, y_pred),
        'ROC-AUC': roc_auc_score(y_true, y_pred),
    }

# Majority voting
test_df['ensemble_vote'] = (test_df[['xg_distil_bert', 'xg_sen_transformer', 'nn_prediction']].mean(axis=1) > 0.5).astype(int)

# Evaluate ensemble performance
ensemble_metrics = calculate_metrics(test_df['actual'], test_df['ensemble_vote'])
print("Ensemble Metrics:", ensemble_metrics)

Ensemble Metrics: {'Accuracy': 0.8960629921259843, 'Precision': 0.9259259259259259, 'Recall': 0.8620689655172413, 'F1-Score': 0.8928571428571429, 'ROC-AUC': 0.8962243561763422}
